In [5]:
import os
import csv

path = 'data/' 

samples = []
with open(path+'/driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader: 
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, test_samples = train_test_split(samples, test_size=.2)
test_samples, valid_samples = train_test_split(samples, test_size=.25)

import numpy as np
import sklearn
import matplotlib.image as mpimg

def generator(samples, batch_size=32):
    while True: 
        shuffle(samples)
        for start in range(0, len(samples), batch_size//2): 
            batch_samples = samples[start:start+batch_size]

            images = []
            angles = []

            for batch_sample in batch_samples:
                name = path+'/IMG/'+batch_sample[0].split('/')[-1]
                center_image = mpimg.imread(name)
                center_angle=float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
                
                # flip image 
                center_image_flip = np.fliplr(center_image)
                center_angle_flip = -center_angle
                images.append(center_image_flip)
                angles.append(center_angle_flip)

            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [12]:
import keras 
import tensorflow as tf

input_shape = (160,320,3)

car_input = keras.layers.Input(shape=input_shape)

cropped_input = keras.layers.Cropping2D(cropping=((50, 25), (0, 0)))(car_input)

crop_input_shape = (85, 300, 3) # new input shape (85, 300, 3)

normal_input = keras.layers.Lambda(lambda x: (x/255.0)-.5)(cropped_input)

resize_factor = 139/85
resize_shape = np.int32(np.multiply(crop_input_shape,resize_factor))[:2]
print(resize_shape)

resized_input = keras.layers.Lambda(lambda image: tf.image.resize_images( 
    image, resize_shape))(normal_input)

[139 490]


In [ ]:
# Loads in InceptionV3
from keras.applications.inception_v3 import InceptionV3


inception = InceptionV3(weights=None, include_top=True,
                        input_shape=(input_size,input_size,3))

In [ ]:

batch_size = 128

train_generator = generator(train_samples, batch_size=batch_size)
valid_generator = generator(validation_samples, batch_size=batch_size)



#model.add(...) 

model.compile(loss='mse', optimizer='Adam')
model.fit_generator(train_generator, /
            steps_per_epoch=ceil(len(train_samples)/batch_size), /
            validation_data=validation_generator, /
            validation_steps=ceil(len(validation_samples)/batch_size), /
            epochs=5, verbose=1)